In [1]:
!sudo salt-key -L --out=json > /mnt/data/salt-keys.json
!sudo salt --static --out json '*' grains.item id os_family os osrelease oscodename osfinger ip4_interfaces kernel > /mnt/data/salt-grains.json


In [2]:
import pandas as pd
import json

tmp = pd.read_json("/mnt/data/salt-grains.json", orient='index')

#https://docs.saltstack.com/en/latest/ref/cli/salt.html
#salt --static --out json '*' grains.item id os_family os osrelease oscodename osfinger ip4_interfaces kernel > /mnt/data/salt-grains.json 
df_grains = pd.read_json(tmp['ret'].to_json(), orient='index')

#usable_dataframe = pd.io.json.json_normalize(jsondata)
#rather roll own?

In [3]:
df_grains

,id,ip4_interfaces,kernel,os,os_family,oscodename,osfinger,osrelease
minion-on-saltmaster,minion-on-saltmaster,"{'lo': ['127.0.0.1'], 'eth0': ['172.17.0.2'], ...",Linux,Ubuntu,Debian,xenial,Ubuntu-16.04,16.04
saltminion01,saltminion01,"{'lo': ['127.0.0.1'], 'eth0': ['172.76.4.3']}",Linux,Ubuntu,Debian,xenial,Ubuntu-16.04,16.04
saltminion02,saltminion02,"{'lo': ['127.0.0.1'], 'eth0': ['172.76.4.4']}",Linux,Debian,Debian,jessie,Debian-8,8.60


In [4]:
import json
from pprint import pprint

with open('/mnt/data/salt-grains.json') as json_data:
    d = json.load(json_data)
    json_data.close()
    pprint(d)

{'minion-on-saltmaster': {'jid': '20170104072355828404',
                          'out': 'nested',
                          'ret': {'id': 'minion-on-saltmaster',
                                  'ip4_interfaces': {'eth0': ['172.17.0.2'],
                                                     'eth1': ['172.76.4.2'],
                                                     'lo': ['127.0.0.1']},
                                  'kernel': 'Linux',
                                  'os': 'Ubuntu',
                                  'os_family': 'Debian',
                                  'oscodename': 'xenial',
                                  'osfinger': 'Ubuntu-16.04',
                                  'osrelease': '16.04'},
                          'retcode': 0},
 'saltminion01': {'jid': '20170104072355828404',
                  'out': 'nested',
                  'ret': {'id': 'saltminion01',
                          'ip4_interfaces': {'eth0': ['172.76.4.3'],
                            

In [5]:
import json

with open('/mnt/data/salt-grains.json') as json_data:
    grains_incoming = json.load(json_data)
    json_data.close()

grains_staged = {}
for key, value in grains_incoming.items():
    retval = {}
    retval['id'] = value['ret']['id']
    retval['kernel'] = value['ret']['kernel']
    retval['os'] = value['ret']['os']
    retval['os_family'] = value['ret']['os_family']
    retval['oscodename'] = value['ret']['oscodename']
    retval['osrelease'] = value['ret']['osrelease']
    retval['osfinger'] = value['ret']['osfinger']
    tmp = []
    for key2, value2 in value['ret']['ip4_interfaces'].items():
        if key2 not in ['lo']:
            tmp.append(value2[0])
    retval['ip4_interfaces'] = ", ".join(tmp)
    grains_staged[key] = retval
grains_staged

{'minion-on-saltmaster': {'id': 'minion-on-saltmaster',
  'ip4_interfaces': '172.17.0.2, 172.76.4.2',
  'kernel': 'Linux',
  'os': 'Ubuntu',
  'os_family': 'Debian',
  'oscodename': 'xenial',
  'osfinger': 'Ubuntu-16.04',
  'osrelease': '16.04'},
 'saltminion01': {'id': 'saltminion01',
  'ip4_interfaces': '172.76.4.3',
  'kernel': 'Linux',
  'os': 'Ubuntu',
  'os_family': 'Debian',
  'oscodename': 'xenial',
  'osfinger': 'Ubuntu-16.04',
  'osrelease': '16.04'},
 'saltminion02': {'id': 'saltminion02',
  'ip4_interfaces': '172.76.4.4',
  'kernel': 'Linux',
  'os': 'Debian',
  'os_family': 'Debian',
  'oscodename': 'jessie',
  'osfinger': 'Debian-8',
  'osrelease': '8.6'}}

In [6]:
import json

with open('/mnt/data/salt-keys.json') as json_data:
    keys_incoming = json.load(json_data)
    json_data.close()

keys_staged={}
for k, col in keys_incoming.items():
    print(col)
    for m in map(str, col):
        keys_staged[m] = k
keys_staged

[]
['T500-2017', 'W7-minion']
[]
['minion-on-saltmaster', 'saltminion01', 'saltminion02']


{'T500-2017': 'minions_pre',
 'W7-minion': 'minions_pre',
 'minion-on-saltmaster': 'minions',
 'saltminion01': 'minions',
 'saltminion02': 'minions'}

In [7]:
from pprint import pprint

minions = {}
for k, v in keys_staged.items():
    tmp = {}
    tmp['keystatus'] = v
    if k in grains_staged:
        tmp['grains'] = grains_staged[k]
    minions[k] = tmp
pprint(minions)

{'T500-2017': {'keystatus': 'minions_pre'},
 'W7-minion': {'keystatus': 'minions_pre'},
 'minion-on-saltmaster': {'grains': {'id': 'minion-on-saltmaster',
                                     'ip4_interfaces': '172.17.0.2, 172.76.4.2',
                                     'kernel': 'Linux',
                                     'os': 'Ubuntu',
                                     'os_family': 'Debian',
                                     'oscodename': 'xenial',
                                     'osfinger': 'Ubuntu-16.04',
                                     'osrelease': '16.04'},
                          'keystatus': 'minions'},
 'saltminion01': {'grains': {'id': 'saltminion01',
                             'ip4_interfaces': '172.76.4.3',
                             'kernel': 'Linux',
                             'os': 'Ubuntu',
                             'os_family': 'Debian',
                             'oscodename': 'xenial',
                             'osfinger': 'Ubuntu

In [8]:
df_minions = pd.DataFrame.from_dict(minions, orient='index')
df_minions

,grains,keystatus
T500-2017,NaN,minions_pre
W7-minion,NaN,minions_pre
minion-on-saltmaster,"{'osfinger': 'Ubuntu-16.04', 'oscodename': 'xe...",minions
saltminion01,"{'osfinger': 'Ubuntu-16.04', 'oscodename': 'xe...",minions
saltminion02,"{'osfinger': 'Debian-8', 'oscodename': 'jessie...",minions
